## Procesamiento

In [101]:
import funciones_comunes
import funciones_mongo 
import funciones_neo4j 
import importlib
importlib.reload(funciones_comunes) # recarga el módulo después de modificarl
importlib.reload(funciones_mongo) # recarga el módulo después de modificarl
importlib.reload(funciones_neo4j)


<module 'funciones_neo4j' from '/work/funciones_neo4j.py'>

In [ ]:
# info = {"usuarios" : [], "destinos" : [], "hoteles": [], "actividades":[], "reservas":[] }
info = funciones_comunes.procesar_datos()
print(info)


## | MongoDB: |

In [52]:
db, estado_mongo = funciones_mongo.iniciar_mongo()

In [26]:
funciones_mongo.insertar_varios_documentos("usuarios",info["usuarios"])

In [27]:
funciones_mongo.insertar_varios_documentos("destinos",info["destinos"]) 

In [28]:
funciones_mongo.insertar_varios_documentos("hoteles",info["hoteles"]) 

In [29]:
funciones_mongo.insertar_varios_documentos("actividades",info["actividades"]) 

In [30]:
funciones_mongo.insertar_varios_documentos("reservas",info["reservas"]) 

In [53]:
for elem in funciones_mongo.db.hoteles.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.destinos.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.usuarios.find():
    print(elem)
print('')
for elem in funciones_mongo.db.actividades.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.reservas.find():
    print(elem) 

{'_id': ObjectId('68f64a72a1ec4ebfdb26aa51'), 'hotel_id': 1, 'nombre': 'Hotel Sol', 'ciudad': 'Bariloche', 'precio': 85000, 'calificacion': 4, 'servicios': ['wifi', 'pileta', 'desayuno']}
{'_id': ObjectId('68f64a72a1ec4ebfdb26aa52'), 'hotel_id': 2, 'nombre': 'Cumbres Andinas', 'ciudad': 'Bariloche', 'precio': 120000, 'calificacion': 5, 'servicios': ['wifi', 'spa', 'pileta']}
{'_id': ObjectId('68f64a72a1ec4ebfdb26aa53'), 'hotel_id': 3, 'nombre': 'Altos del Norte', 'ciudad': 'Jujuy', 'precio': 60000, 'calificacion': 3, 'servicios': ['wifi']}
{'_id': ObjectId('68f64a72a1ec4ebfdb26aa54'), 'hotel_id': 4, 'nombre': 'Montaña Real', 'ciudad': 'Mendoza', 'precio': 95000, 'calificacion': 4, 'servicios': ['wifi', 'pileta']}
{'_id': ObjectId('68f64a72a1ec4ebfdb26aa55'), 'hotel_id': 5, 'nombre': 'Estancia Colonial', 'ciudad': 'Córdoba', 'precio': 70000, 'calificacion': 4, 'servicios': ['wifi', 'desayuno']}

{'_id': ObjectId('68f64a72a1ec4ebfdb26aa4c'), 'destino_id': 1, 'ciudad': 'Bariloche', 'pais'

Base de datos:

viajes ← (esa es la variable db) 
Colecciones dentro de viajes:

usuarios

destinos

hoteles

actividades

reservas

Ejemplo visual:
viajes
 ├── usuarios
 │    ├── { "usuario_id": 1, "nombre": "Juan", "email": "juan@mail.com" }
 │    └── { "usuario_id": 2, "nombre": "Ana", "email": "ana@mail.com" }
 ├── destinos
 │    ├── { "destino_id": 1, "nombre": "Roma" }
 │    └── { "destino_id": 2, "nombre": "Tokio" }
 ├── hoteles
 │    └── { "hotel_id": 1, "nombre": "Hilton", "destino_id": 1 }
 ├── actividades
 │    └── { "actividad_id": 1, "nombre": "Tour guiado", "destino_id": 1 }
 └── reservas
      └── { "reserva_id": 1, "usuario_id": 1, "hotel_id": 1, "fecha": "2025-10-18" }

In [23]:
def drop_dataset():
    lista = ["usuarios","destinos","hoteles","actividades","reservas"]
    for elem in lista:
        db[elem].drop() # Elimina por completo la colección destinos dentro de la base de datos viajes.

In [24]:
drop_dataset()

## | Neo4J: |

In [127]:
from neo4j import GraphDatabase
import os
from funciones_neo4j import insertar_varios_nodos,insertar_varias_relaciones
from funciones_comunes import obtener_datos_mongo

# Datos de conexión desde variables de entorno
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4j123")
NEO4J_USER = "neo4j"  # usuario por defecto en el contenedor
NEO4J_URI = "bolt://neo4j:7687"  # host del contenedor de Neo4j

# Crear driver de conexión
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [92]:
insertar_varios_nodos("Usuario", info["usuarios"])
insertar_varios_nodos("Destino", info["destinos"])

In [191]:
def crear_relacion(rel,receptor="Usuario"):
    relacion = info[rel] 
    prop = f"{receptor.lower()}_id"
    print(prop)
    lista_relaciones = [
        {
            "label_origen": "Usuario",
            "prop_origen": "usuario_id",
            "valor_origen": r["origen_id"],
            "label_destino": receptor,
            "prop_destino": prop, 
            "valor_destino": r["receptor_id"],
            "tipo": rel.upper()
        }
        for r in relacion
    ]
    insertar_varias_relaciones(lista_relaciones)

In [189]:
relacion = info["amigo"] 
lista_relaciones = [
    {
        "label_origen": "Usuario",
        "prop_origen": "usuario_id",
        "valor_origen": r["origen_id"],
        "label_destino": "Usuario",
        "prop_destino": "usuario_id", 
        "valor_destino": r["receptor_id"],
        "tipo": "amigo"
    }
    for r in relacion
]
print(lista_relaciones)
insertar_varias_relaciones(lista_relaciones)

[{'label_origen': 'Usuario', 'prop_origen': 'usuario_id', 'valor_origen': 1, 'label_destino': 'Usuario', 'prop_destino': 'usuario_id', 'valor_destino': 2, 'tipo': 'amigo'}, {'label_origen': 'Usuario', 'prop_origen': 'usuario_id', 'valor_origen': 2, 'label_destino': 'Usuario', 'prop_destino': 'usuario_id', 'valor_destino': 1, 'tipo': 'amigo'}]


In [192]:
crear_relacion("visito",receptor="Destino")
crear_relacion("amigo")
crear_relacion("familiar")

destino_id
usuario_id
usuario_id


In [121]:
## Para mostar las relaciones 
def print_relaciones(relacion):
    
    query_1 = f'MATCH (u:Usuario)-[r:{relacion}]->(d:Destino) RETURN count(r)'
    query_2 = f"MATCH (u:Usuario)-[r:{relacion}]->(d:Destino) RETURN u.usuario_id, d.destino_id "
            
    with driver.session() as session:
        # Contar Relaciones
        resultado = session.execute_read( lambda tx: tx.run(query_1).values() )
        print(f'Total relaciones {relacion} : {resultado}')

        # Mostrar Relaciones
        relaciones = session.execute_read( lambda tx: tx.run(query_2).data() )
        print(relaciones)

In [196]:
print_relaciones("VISITO")
print_relaciones("AMIGO")
print_relaciones("FAMILIAR")

Total relaciones VISITO : [[7]]
[{'u.usuario_id': 1, 'd.destino_id': 5}, {'u.usuario_id': 1, 'd.destino_id': 1}, {'u.usuario_id': 2, 'd.destino_id': 1}, {'u.usuario_id': 3, 'd.destino_id': 3}, {'u.usuario_id': 4, 'd.destino_id': 2}, {'u.usuario_id': 5, 'd.destino_id': 4}, {'u.usuario_id': 5, 'd.destino_id': 1}]
Total relaciones AMIGO : [[0]]
[]
Total relaciones FAMILIAR : [[0]]
[]


In [176]:
# Eleminiar todas las relacioens
with driver.session() as session:
    session.execute_write(
        lambda tx: tx.run("MATCH ()-[r]->() DELETE r")
    )

## | Redis :| 

In [45]:
import redis
import os 
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "redis123")
r = redis.Redis(host="redis", port=6379, password=REDIS_PASSWORD, decode_responses=True)

In [4]:
## simulamos que hay 3 usuarios online 
## Usuario1 Usuario2 y Usuario3 

def get_usuarios_conectados(): 
    us = [("usuario_id",1),("usuario_id",2),("usuario_id",3)] 
    return us 
conectados = get_usuarios_conectados()

for usuarios in conectados: 
     r.sadd("presence:online", (f'usuario_id:{usuarios[1]}'))
r.expire("presence:online", 1800) # expira en 30 minutos

True

In [66]:
## Guardando reservas inconclusas
for elem in (db.reservas.find({"estado":{"$ne":"Confirmada"}})):
    elem["_id"] = str(elem["_id"])
    
    key = f'reserva_id:{elem["reserva_id"]}'
    r.sadd("reservas:inconclusas", key) 
    r.hset(key,mapping=elem) 

{'_id': '68f64a73a1ec4ebfdb26aa5c', 'reserva_id': '2', 'usuario_id': '2', 'destino_id': '1', 'fecha_reserva': '2025-06-15', 'estado': 'Pagada', 'precio_total': '90000'}
{'_id': '68f64a73a1ec4ebfdb26aa5e', 'reserva_id': '4', 'usuario_id': '1', 'destino_id': '4', 'fecha_reserva': '2025-07-10', 'estado': 'Pendiente', 'precio_total': '100000'}
{'_id': '68f64a73a1ec4ebfdb26aa5d', 'reserva_id': '3', 'usuario_id': '3', 'destino_id': '3', 'fecha_reserva': '2025-05-20', 'estado': 'Cancelada', 'precio_total': '110000'}


In [ ]:
for key in r.smembers("reservas:inconclusas"):
    datos = r.hgetall(key)
    print(datos)

## | Consultas: |

### Funciones Auxiliares

In [169]:
def funcion_query_1(query,query_name):
    import json 

    cache_key = f"cache:{query_name}"
    # Consulta si existe en redis
    if r.exists(cache_key):
        print("🟢 Obtenido desde Redis")
        return json.loads(r.get(cache_key))

    with driver.session() as session: 
        resultado = session.execute_read ( 
            lambda tx: tx.run ( query ).values()
        )
    r.setex(cache_key,450,json.dumps(resultado)) 

    print("🟢 Obtenido desde Neo4j")
    return resultado
    
def funcion_query_2(query,query_name):
    import json 

    cache_key = f"cache:{query_name}"
    # Consulta si existe en redis
    if r.exists(cache_key):
        print("🟢 Obtenido desde Redis")
        return json.loads(r.get(cache_key))

    with driver.session() as session: 
        resultado = session.execute_read ( 
            lambda tx: tx.run ( query ).data()
        )
    r.setex(cache_key,450,json.dumps(resultado)) 

    print("🟢 Obtenido desde Neo4j")
    return resultado

### a. Mostrar los usuarios que visitaron “Bariloche”. 

In [167]:
query = "MATCH (u:Usuario)-[:visito]->(d:Destino {destino_id: 1}) RETURN u.nombre"
print(funcion_query_1(query, "busqueda:lga"))

🟢 Obtenido desde Neo4j
[['Ana Torres'], ['Juan López'], ['María Pérez']]


### b. Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [174]:
query = "MATCH (j:Usuario {nombre: 'Juan López' })-[:amigo]->(a:Usuario) RETURN a "
print(funcion_query_2(query,"amigos:visitaazk")) 

🟢 Obtenido desde Neo4j
[]


In [ ]:
, (j)-[:visito]->(d:Destino), (a)-[:visito]->(d) RETURN a.nombre , d.ciudad "